In [1]:
from model import Posicao, Extrato
from views import FundoInvestimento, Acao, FundoImobiliario
#from controllers import MainController
import numpy as np
import pandas as pd 
from datetime import datetime

import plotly.graph_objs as go

# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
posicao_model = Posicao() 
posicao_model.load_data()
extrato = Extrato(2010, 2020)
fundo_investimento = FundoInvestimento(
    posicao=posicao_model.fis, 
    extrato=extrato.extrato_fis
)
acoes = Acao(posicao=posicao_model.acoes, extrato=extrato.extrato_acoes)
fiis = FundoImobiliario(posicao=posicao_model.fiis, extrato=extrato.extrato_fiis)

/Users/gabriellopes/Documents/Pessoal/dev/finance-app/app/classes/model.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.__aportes_fi_hist['Nome'] = self.__aportes_fi_hist['Descricao'].apply(self.__map_fi)
/Users/gabriellopes/Documents/Pessoal/dev/finance-app/app/classes/model.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ir_fi_hist['Nome'] = self.ir_fi_hist['Descricao'].apply(self.__map_fi)


**Mapeado todos os FIs**

Verifica se existe FI no extrato não mapeado

In [3]:
#fundo_investimento.extrato[fundo_investimento.extrato['Nome'].str.contains('unknown')].head()
#extrato.aportes_fi_hist[extrato.aportes_fi_hist['Nome'] == 'unknown']

## Analise FIs

In [4]:
resumo_ = fundo_investimento.resumo

resumo_.head()

,Nome,Total Bruto,mes,ano,data_posicao,index,aporte,retirada,rendimento_resgatado,rendimento,periodo_cont
0,AZ Quest Multi FIC FIM,0.00,1,2010,2010-01-31,0.00,0.00,0.00,0.00,0.00,0.00
1,AZ Quest Multi FIC FIM,0.00,2,2010,2010-02-28,1.00,0.00,0.00,0.00,0.00,0.00
2,AZ Quest Multi FIC FIM,0.00,3,2010,2010-03-31,2.00,0.00,0.00,0.00,0.00,0.00
3,AZ Quest Multi FIC FIM,0.00,4,2010,2010-04-30,3.00,0.00,0.00,0.00,0.00,0.00
4,AZ Quest Multi FIC FIM,0.00,5,2010,2010-05-31,4.00,0.00,0.00,0.00,0.00,0.00


In [5]:
# um fundo especifico

#display(resumo_[resumo_['Nome'] == 'Azul Quantitativo FIM'])

In [6]:
tmp = resumo_.groupby(['Nome'])\
        .agg(total_aporte=('aporte', 'sum'), 
             total_rendimento=('rendimento', 'sum'), 
             total_retirada=('retirada', 'sum')).reset_index()

tmp

,Nome,total_aporte,total_rendimento,total_retirada
0,AZ Quest Multi FIC FIM,"15,000.00","3,591.84","18,591.84"
1,Azul Quantitativo FIM,"15,200.00","2,624.47","17,824.47"
2,BNP Paribas Inflação FI RF,"10,300.00",-131.86,"10,168.14"
3,Bahia AM Maraú FIC de FIM,"73,000.00","12,897.97",0.00
4,Equitas Selection FIC FIA,"19,900.00","-1,156.14",0.00
5,Icatu Vanguarda Pré-Fixado FIRF,"31,700.00",-934.23,"30,765.77"
6,Mauá Macro FIC FIM,"23,500.00","5,382.28",0.00
7,Mirae Asset Multimercado,"16,000.00","3,122.71","19,122.71"
8,NP Hedge FIC FIM,"5,000.00",144.33,"5,144.33"
9,Polo Norte I FIC FIM,"30,000.00","3,556.88",0.00


In [7]:
display(resumo_[resumo_['Nome'] == 'BNP Paribas Inflação FI RF'])

,Nome,Total Bruto,mes,ano,data_posicao,index,aporte,retirada,rendimento_resgatado,rendimento,periodo_cont
0,BNP Paribas Inflação FI RF,0.00,1,2010,2010-01-31,264.00,0.00,0.00,0.00,0.00,0.00
1,BNP Paribas Inflação FI RF,0.00,2,2010,2010-02-28,265.00,0.00,0.00,0.00,0.00,0.00
2,BNP Paribas Inflação FI RF,0.00,3,2010,2010-03-31,266.00,0.00,0.00,0.00,0.00,0.00
3,BNP Paribas Inflação FI RF,0.00,4,2010,2010-04-30,267.00,0.00,0.00,0.00,0.00,0.00
4,BNP Paribas Inflação FI RF,0.00,5,2010,2010-05-31,268.00,0.00,0.00,0.00,0.00,0.00
5,BNP Paribas Inflação FI RF,0.00,6,2010,2010-06-30,269.00,0.00,0.00,0.00,0.00,0.00
6,BNP Paribas Inflação FI RF,0.00,7,2010,2010-07-31,270.00,0.00,0.00,0.00,0.00,0.00
7,BNP Paribas Inflação FI RF,0.00,8,2010,2010-08-31,271.00,0.00,0.00,0.00,0.00,0.00
8,BNP Paribas Inflação FI RF,0.00,9,2010,2010-09-30,272.00,0.00,0.00,0.00,0.00,0.00
9,BNP Paribas Inflação FI RF,0.00,10,2010,2010-10-31,273.00,0.00,0.00,0.00,0.00,0.00


## Analise Ações

In [8]:
df_extacoes = acoes.extrato
df_posacoes = acoes.posicao
df_extfiis = fiis.extrato
df_posfiis = fiis.posicao

In [9]:
df_extacoes['ano'] = df_extacoes['Data'].dt.year
df_extacoes['mes'] = df_extacoes['Data'].dt.month

In [10]:
resumo, tmp = acoes.calcula_resumo(2010, 2020)

#display(tmp[tmp['Papel'] == 'BPAC11'])

tmp

,index,Papel,ano,mes,Data,Financeiro,aporte,retirada,rendimento,periodo_cont
0,0,BPAC11,2010,1,2010-01-31,0.00,0.00,0.00,0.00,0
1,1,BPAC11,2010,2,2010-02-28,0.00,0.00,0.00,0.00,0
2,2,BPAC11,2010,3,2010-03-31,0.00,0.00,0.00,0.00,0
3,3,BPAC11,2010,4,2010-04-30,0.00,0.00,0.00,0.00,0
4,4,BPAC11,2010,5,2010-05-31,0.00,0.00,0.00,0.00,0
5,5,BPAC11,2010,6,2010-06-30,0.00,0.00,0.00,0.00,0
6,6,BPAC11,2010,7,2010-07-31,0.00,0.00,0.00,0.00,0
7,7,BPAC11,2010,8,2010-08-31,0.00,0.00,0.00,0.00,0
8,8,BPAC11,2010,9,2010-09-30,0.00,0.00,0.00,0.00,0
9,9,BPAC11,2010,10,2010-10-31,0.00,0.00,0.00,0.00,0


In [11]:
#display(df_extacoes.sort_values('Papel'))
display(df_posacoes.sort_values(['Papel', 'data_posicao']))

,Papel,Qtd Disponivel,Qtd Projetado,Qtd Dia,Qtd Garantia BOV,Qtd Garantia BMF,Qtd Estruturados,Liq Termo,Qtd Total,Cotacao,Financeiro,period,mes,ano,data_posicao
36,BPAC11,200,0,0,0,0,0,0,200,33.24,"6,648.00",2020/3,3,2020,2020-03-31
16,BPAC11,200,0,0,0,0,0,0,200,42.30,"8,460.00",2020/4,4,2020,2020-04-30
3,BPAC11,200,0,0,0,0,0,0,200,48.84,"9,768.00",2020/5,5,2020,2020-05-29
59,BPAC11,26,0,0,0,0,0,0,26,86.42,"2,246.92",2020/7,7,2020,2020-07-31
9,BPAC11,26,0,0,0,0,0,0,26,80.33,"2,088.58",2020/8,8,2020,2020-08-31
41,BPAC11,26,0,0,0,0,0,0,26,72.85,"1,894.10",2020/9,9,2020,2020-09-30
21,BPAC11,26,0,0,0,0,0,0,26,72.50,"1,885.00",2020/10,10,2020,2020-10-30
49,BPAC11,26,0,0,0,0,0,0,26,79.22,"2,059.72",2020/11,11,2020,2020-11-30
29,BPAC12,52,0,0,0,0,0,0,52,24.80,"1,289.60",2020/6,6,2020,2020-06-30
30,BPAC9,26,0,0,0,0,0,0,26,24.80,644.80,2020/6,6,2020,2020-06-30


In [12]:
acoes.calcula_resumo(2010, 2020)

(       Papel   ano  mes       Data  Financeiro    aporte  retirada
 0     BPAC11  2010    1 2010-01-31        0.00      0.00      0.00
 1     BPAC11  2010    2 2010-02-28        0.00      0.00      0.00
 2     BPAC11  2010    3 2010-03-31        0.00      0.00      0.00
 3     BPAC11  2010    4 2010-04-30        0.00      0.00      0.00
 4     BPAC11  2010    5 2010-05-31        0.00      0.00      0.00
 5     BPAC11  2010    6 2010-06-30        0.00      0.00      0.00
 6     BPAC11  2010    7 2010-07-31        0.00      0.00      0.00
 7     BPAC11  2010    8 2010-08-31        0.00      0.00      0.00
 8     BPAC11  2010    9 2010-09-30        0.00      0.00      0.00
 9     BPAC11  2010   10 2010-10-31        0.00      0.00      0.00
 10    BPAC11  2010   11 2010-11-30        0.00      0.00      0.00
 11    BPAC11  2010   12 2010-12-31        0.00      0.00      0.00
 12    BPAC11  2011    1 2011-01-31        0.00      0.00      0.00
 13    BPAC11  2011    2 2011-02-28        0.00 

In [13]:
display(df_extfiis.head())
display(df_posfiis.head())

,Papel,Categoria,Tipo,Qtde,Preco,Data
1,BCFF11,FII,COMPRA,58,88.60,2020-12-17
7,XPLG11,FII,COMPRA,50,126.97,2020-12-03
9,BCFF11,FII,COMPRA,300,90.60,2020-10-09
16,BCFF11,FII,COMPRA,46,83.60,2020-05-28
21,OULG11B,FII,COMPRA,50,113.55,2020-01-20


,Papel,Qtd Disponivel,Qtd Projetada,Qtd Dia,Qtde Total,Ult Cotacao,Financeiro,period,mes,ano,data_posicao
0,IBFF11,50,0,0,50.00,100.10,"5,005.00",2020/2,2,2020,2020-02-28
1,BCFF11,8,0,0,8.00,98.17,785.36,2020/2,2,2020,2020-02-28
2,CNES11,11,0,0,11.00,83.50,918.50,2020/2,2,2020,2020-02-28
3,OULG11B,50,0,0,50.00,105.00,"5,250.00",2020/2,2,2020,2020-02-28
4,RNGO11,49,0,0,49.00,96.61,"4,733.89",2020/2,2,2020,2020-02-28


In [14]:
acoes.posicao['Papel'].unique()

array(['JHSF3', 'MGLU3', 'BPAC11', 'ITSA4', 'PETR4', 'WEGE3', 'EGIE3',
       'COGN3', 'BPAC12', 'BPAC9', 'TSLA34'], dtype=object)

In [15]:
fiis.posicao['Papel'].unique()

array(['IBFF11', 'BCFF11', 'CNES11', 'OULG11B', 'RNGO11', 'BCFF11B',
       'CNES11B', 'BCFF12', 'OULG11', 'IBFF12', 'TBOF11', 'HGLG14',
       'HGLG11', 'BCFF12B', 'CNES12B'], dtype=object)

In [16]:
acoes.posicao[acoes.posicao['Papel'] == 'MGLU3'].sort_values('data_posicao')

,Papel,Qtd Disponivel,Qtd Projetado,Qtd Dia,Qtd Garantia BOV,Qtd Garantia BMF,Qtd Estruturados,Liq Termo,Qtd Total,Cotacao,Financeiro,period,mes,ano,data_posicao
2,MGLU3,800,0,0,0,0,0,0,800,37.04,"29,632.00",2019/9,9,2019,2019-09-30
65,MGLU3,800,0,0,0,0,0,0,800,44.64,"35,712.00",2019/10,10,2019,2019-10-31
68,MGLU3,800,0,0,0,0,0,0,800,45.03,"36,024.00",2019/11,11,2019,2019-11-29
58,MGLU3,800,0,0,0,0,0,0,800,47.66,"38,128.00",2019/12,12,2019,2019-12-31
67,MGLU3,800,0,0,0,0,0,0,800,55.80,"44,640.00",2020/1,1,2020,2020-01-31
1,MGLU3,800,0,0,0,0,0,0,800,50.52,"40,416.00",2020/2,2,2020,2020-02-28
38,MGLU3,800,0,0,0,0,0,0,800,38.99,"31,192.00",2020/3,3,2020,2020-03-31
18,MGLU3,800,0,0,0,0,0,0,800,49.70,"39,760.00",2020/4,4,2020,2020-04-30
6,MGLU3,800,0,0,0,0,0,0,800,64.35,"51,480.00",2020/5,5,2020,2020-05-29
33,MGLU3,800,0,0,0,0,0,0,800,71.65,"57,320.00",2020/6,6,2020,2020-06-30


In [17]:
acoes.posicao.sort_values(['Papel', 'ano', 'mes'])

,Papel,Qtd Disponivel,Qtd Projetado,Qtd Dia,Qtd Garantia BOV,Qtd Garantia BMF,Qtd Estruturados,Liq Termo,Qtd Total,Cotacao,Financeiro,period,mes,ano,data_posicao
36,BPAC11,200,0,0,0,0,0,0,200,33.24,"6,648.00",2020/3,3,2020,2020-03-31
16,BPAC11,200,0,0,0,0,0,0,200,42.30,"8,460.00",2020/4,4,2020,2020-04-30
3,BPAC11,200,0,0,0,0,0,0,200,48.84,"9,768.00",2020/5,5,2020,2020-05-29
59,BPAC11,26,0,0,0,0,0,0,26,86.42,"2,246.92",2020/7,7,2020,2020-07-31
9,BPAC11,26,0,0,0,0,0,0,26,80.33,"2,088.58",2020/8,8,2020,2020-08-31
41,BPAC11,26,0,0,0,0,0,0,26,72.85,"1,894.10",2020/9,9,2020,2020-09-30
21,BPAC11,26,0,0,0,0,0,0,26,72.50,"1,885.00",2020/10,10,2020,2020-10-30
49,BPAC11,26,0,0,0,0,0,0,26,79.22,"2,059.72",2020/11,11,2020,2020-11-30
29,BPAC12,52,0,0,0,0,0,0,52,24.80,"1,289.60",2020/6,6,2020,2020-06-30
30,BPAC9,26,0,0,0,0,0,0,26,24.80,644.80,2020/6,6,2020,2020-06-30
